In [1]:
import torch

In [2]:
import torch
print(torch.cuda.is_available())


True


In [3]:
import pandas as pd

In [41]:
train_dataset=pd.read_csv("/content/fashion-mnist_train.csv")

In [42]:
train_dataset.shape

(60000, 785)

In [98]:
train_features=train_dataset.drop(columns=["label"]).values
train_labels=train_dataset["label"].values

In [99]:
train_features=train_features/255.0

In [100]:
test_dataset=pd.read_csv("/content/fashion-mnist_test.csv")

In [101]:
test_features=test_dataset.drop(columns=["label"]).values
test_labels=test_dataset["label"].values

In [102]:
test_features=test_features/255.0

In [103]:
test_dataset.shape

(10000, 785)

In [104]:
from torch.utils.data import Dataset,DataLoader

In [105]:
class customdataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [106]:
train_datasetss=customdataset(train_features,train_labels)
test_datasetss=customdataset(test_features,test_labels)

In [107]:
import numpy as np
np.unique(train_labels)





array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [108]:
train_labels.shape

(60000,)

In [109]:
len(train_datasetss)

60000

In [110]:
len(test_datasetss)

10000

In [111]:
train_datasetss[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0157, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2431, 0.2392, 0.0824, 0.1137, 0.0902,
         0.2000, 0.5333, 0.2392, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [136]:
train_dataloader=DataLoader(train_datasetss,batch_size=32,shuffle=True,pin_memory=True)
test_dataloader=DataLoader(test_datasetss,batch_size=32,shuffle=False,pin_memory=True)


In [113]:
len(train_dataloader)## it return no of batch formed

1875

In [114]:
import torch.nn as nn

In [137]:
## create model
class models(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.network=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,features):
    out=self.network(features)
    return out

In [116]:
epochs=100
learning_rate=0.0001

In [117]:
model=models(train_features.shape[1])


In [119]:
## move to gpu first step is
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [120]:
model=model.to(device)

In [121]:
## intialize loss function
criterion=nn.CrossEntropyLoss()

In [141]:
## OPtmizers
import torch.optim as optim

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [139]:
for batch_features, batch_labels in train_dataloader:
    print("Batch features:", batch_features.shape)
    print("Batch labels:", batch_labels.shape, batch_labels.dtype)
    print("Unique labels in this batch:", torch.unique(batch_labels))
    break


Batch features: torch.Size([32, 784])
Batch labels: torch.Size([32]) torch.int64
Unique labels in this batch: tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])


In [142]:
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features,batch_labels in train_dataloader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    output=model(batch_features)
    loss=criterion(output,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss+=loss.item()
  avg_epoch_loss=(total_epoch_loss/len(train_dataloader))
  print(f'epoch:{epoch+1},loss:{avg_epoch_loss}')

epoch:1,loss:0.4791564213792483
epoch:2,loss:0.3739503669897715
epoch:3,loss:0.33729387341539063
epoch:4,loss:0.3162149763941765
epoch:5,loss:0.30069934588273367
epoch:6,loss:0.2879052993675073
epoch:7,loss:0.27520094324747724
epoch:8,loss:0.26354894472857315
epoch:9,loss:0.25312624922792115
epoch:10,loss:0.2454974726875623
epoch:11,loss:0.2364864946603775
epoch:12,loss:0.2305101901859045
epoch:13,loss:0.22146322190860906
epoch:14,loss:0.2170995262513558
epoch:15,loss:0.2094918011367321
epoch:16,loss:0.2045783480644226
epoch:17,loss:0.1992534518164893
epoch:18,loss:0.19261744217673937
epoch:19,loss:0.18776138822883368
epoch:20,loss:0.1834869083603223
epoch:21,loss:0.18023780127714079
epoch:22,loss:0.17722119673391185
epoch:23,loss:0.1685527696321408
epoch:24,loss:0.16732690485815208
epoch:25,loss:0.16178067205039162
epoch:26,loss:0.16098179706657925
epoch:27,loss:0.1576891709819436
epoch:28,loss:0.15439804903318485
epoch:29,loss:0.14883737905919553
epoch:30,loss:0.14793141118387382
epo

In [ ]:
## model evaluation

In [143]:
model.eval()

models(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [144]:
total=0
correct=0
with torch.no_grad():

  for batch_features,batch_labels in test_dataloader:

    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    output=model(batch_features)
    _, predicted=torch.max(output,1)

    total+=batch_labels.shape[0]
    correct+=(predicted==batch_labels).sum().item()
  print(correct/total)

0.8966


In [ ]:
## ened